## Part 1

### 01. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### 02. Importing the dataset

In [2]:
#create a path
path = r'C:\Users\Lumex\Documents\Achievement_4_Project\InstaCart Basket Analysis'
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [3]:
#checking the dataframe that was imported
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
df.shape

(206209, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### 03. Wrangle the data so that it follows consistent logic

In [6]:
# rename the column names to be uniform
df = df.rename(columns = { 
    'user_id' : 'user_id',
    'First Name' : 'first_name',
    'Surnam' : 'last_name',
    'STATE' : 'state',
    'Age' : 'age',
    'date_joined' : 'date_joined', 
    'n_dependants' : 'number_of_dependents', 
    'fam_status' : 'marital_status'})

In [7]:
# Checking if the column names are now uniform
df.columns

Index(['user_id', 'first_name', 'last_name', 'Gender', 'state', 'age',
       'date_joined', 'number_of_dependents', 'marital_status', 'income'],
      dtype='object')

I do not have any reason to drop any column from the dataframe.

### 04. Fundamental data quality and consistency checks

In [8]:
df[['age','number_of_dependents','income']].describe()

,age,number_of_dependents,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [9]:
# checking for missing values
missing_values = df.isnull().sum()

In [10]:
missing_values

user_id                     0
first_name              11259
last_name                   0
Gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependents        0
marital_status              0
income                      0
dtype: int64

There are 11,259 missing values in the first name column. It does not affect the analysis because the first name is not needed for the analysis.

In [11]:
# Checking for duplicates in the dataframe
df_dups = df[df.duplicated()]

In [12]:
df_dups

,user_id,first_name,last_name,Gender,state,age,date_joined,number_of_dependents,marital_status,income


No duplicates was found in the dataframe.

In [13]:
#understanding the column data type
df.dtypes

user_id                  int64
first_name              object
last_name               object
Gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependents     int64
marital_status          object
income                   int64
dtype: object

There is no mixed-type data found in the columns

### 05. Combine the customer data with the rest of your prepared Instacart data

In [14]:
# Import the rest of prepared Instacart data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'aggregated_ords_prods_merged.pkl'))

In [15]:
# check the datatype of ords_prods_merge dataframe
ords_prods_merge.dtypes

Unnamed: 0                   int64
order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
_merge                    category
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
match                     category
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_spend              float64
spender_flag                object
customer_regularity        float64
frequency_flag              object
dtype: object

The key column(user_id) for the dataframes have the same datatype.

In [16]:
# Drop the '_merge' column in the ords_prods_merge dataframe
ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])

In [17]:
#check if the _merge column is removed
ords_prods_merge.columns

Index(['Unnamed: 0', 'order_id', 'user_id', 'order_number',
       'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'match', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_spend', 'spender_flag', 'customer_regularity',
       'frequency_flag'],
      dtype='object')

In [ ]:
#Merge ords_prods_merge with customers data
df_merged_new = ords_prods_merge.merge(df, on = 'user_id', indicator = True)

In [ ]:
df_merged_new.head()

In [ ]:
# Export the new dataframe as a pickle file
df_merged_new.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customer_aggregated.pkl'))